<a href="https://colab.research.google.com/github/dlackner/CCBT/blob/master/Software_Guide_Senior_Design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes for Senior Design

## Installing MAVROS on Jetson TX1

> The mavros ROS package enables MAVLink extendable communication between computers running ROS, MAVLink enabled autopilots, and MAVLink enabled GCS.
*   Developers working with ROS are advised to use the source installation rather than binary installation, so we installed from source. [Installation Instructions](https://dev.px4.io/en/ros/mavros_installation.html)
*   `catkin build` failed on our Jetson TX1 (most likely a result of running out of memory), but worked when we tried building with two cores instead.
```
catkin build -j2
```
* Don't use both `catkin_make` and `catkin build` in the same workspace. Their build artifacts are incompatible. [Source](https://answers.ros.org/question/277109/catkin_make-works-but-catkin-build-fails/)
* `catkin config` will tell you which toolset is currently in use. If you wish to switch to `catkin build`, run 
```
catkin clean -bdy
```

## Connecting Jetson TX1 to Pixhawk

> A simple way to connect Jetson TX1 and Pixhawk is using the **UART (J17) connector** which provides serial interface. This interface is accessed via the **/dev/ttyTHS2** device in Linux. The UART connector and Pixhawk were connected in the following way:
> * Pin 1 J17 (GND) to pin 6 (GND) on Pixhawk TELEM2 port.
* Pin 4 J17 (RX) to pin 2 (TX) on Pixhawk TELEM2 port.
* Pin 5 J17 (TX) to pin 3 (RX) on Pixhawk TELEM2 port.

> [Source](https://github.com/NVIDIA-AI-IOT/redtail/wiki/Dev-board-UART-and-Pixhawk-connection)
<br>
> [Pixhawk Pinout](#scrollTo=7W5RGjEyi659)


## Launching the MAVROS Node

> With the UART connection, the FCU source device is **/dev/ttyTHS2**, not **/dev/ttyACM0** in the default `apm.launch`. In addition, the baud rate should be set to 921600. Therefore, the custom launch file was made. Launch MAVROS node with the following customized launch file with
```
roslaunch mavros jetson_apm.launch
```

In [0]:
%%xml
<launch>
	<!-- vim: set ft=xml noet : -->
	<!-- example launch script for ArduPilotMega based FCU's -->

	<arg name="fcu_url" default="/dev/ttyTHS2:921600" />
	<arg name="gcs_url" default="" />
	<arg name="tgt_system" default="1" />
	<arg name="tgt_component" default="1" />
	<arg name="log_output" default="screen" />
	<arg name="fcu_protocol" default="v2.0" />
	<arg name="respawn_mavros" default="false" />

	<include file="$(find mavros)/launch/node.launch">
		<arg name="pluginlists_yaml" value="$(find mavros)/launch/apm_pluginlists.yaml" />
		<arg name="config_yaml" value="$(find mavros)/launch/apm_config.yaml" />

		<arg name="fcu_url" value="$(arg fcu_url)" />
		<arg name="gcs_url" value="$(arg gcs_url)" />
		<arg name="tgt_system" value="$(arg tgt_system)" />
		<arg name="tgt_component" value="$(arg tgt_component)" />
		<arg name="log_output" value="$(arg log_output)" />
		<arg name="fcu_protocol" value="$(arg fcu_protocol)" />
		<arg name="respawn_mavros" default="$(arg respawn_mavros)" />
	</include>
</launch>

## Launching the RealSense Node

> We were unable to install the ROS Wrapper for Intel RealSense Devices using instructions from their [Github](https://github.com/intel-ros/realsense), so we downloaded the realsense2_camera folder into `~/catkin_ws/src/` and ran 
```
catkin build -j2
```
This allowed us to add the package to the workspace. To start the camera node in ROS, install rgbd_launch:
```
sudo apt-get install ros-kinetic-rgbd-launch
```
To launch the node:
```
roslaunch realsense2_camera rs_aligned_depth.launch
```
To view image topics, run the following command and select the topic you want to view
```
rqt_image_view
```

## Startup Guide

> In a terminal window, launch roscore
```
roscore
```
In another terminal window, launch the RealSense camera node (for camera topics):
```
roslaunch realsense2_camera rs_aligned_depth.launch
```
In a third terminal window, launch the MAVROS node (for mavros topics, not necessary for testing camera)
```
roslaunch mavros jetson_apm.launch
```
Then, in a fourth terminal window, run your script. 

### Notes

* **Launching nodes with ROS**<br>
```
roslaunch [package_name] [launch_file]
```
[Source](https://github.com/intel-aero/meta-intel-aero/wiki/05-Autonomous-drone-programming-with-ROS)
* To view available topics
```
rostopic list
```
* Use Guided mode for autonomous. Switch to Hold mode for arming and disarming.
* TM: Wrong FCU time` because no GPS
* [Correct Velocity Commands](http://forum.erlerobotics.com/t/help-with-rover-guided-mode-not-working-with-mavros/1453/2)
* [Tuning Speed/Throttle](http://ardupilot.org/rover/docs/rover-tuning-throttle-and-speed.html)
* [Switch to Guided Mode](https://discuss.ardupilot.org/t/rover-in-guide-mode/26919/8)
* [Start in Guided](https://discuss.ardupilot.org/t/driving-rover-with-waypoints-without-gps/29285)<br>
In QGroundControl, set INITIAL_MODE to 15, and also set flight mode 4 to be GUIDED so you can simply flip on switch on your RC transmitter.
* **Might need to customize firmware?**
* https://github.com/ArduPilot/ardupilot/blob/master/APMrover2/GCS_Mavlink.cpp
* https://github.com/ArduPilot/ardupilot/blob/master/APMrover2/RC_Channel.cpp
* https://github.com/ArduPilot/ardupilot/blob/master/APMrover2/Rover.h
* https://github.com/ArduPilot/ardupilot/blob/master/APMrover2/mode.h
* https://github.com/ArduPilot/ardupilot/blob/master/APMrover2/mode_guided.cpp


# Appendix

## Pixhawk Pinout
<img src="https://drive.google.com/uc?export=download&id=16HfNRwXV7mjNjBLkNoK16FaVc7CTtgPm"/>


# Software Notes

**Software Architecture Plan**

*   Visual Input Stream
*   Publish stream to ROS package
*   Convert stream to OpenCV image Mat (cv_bridge)
*   Detect goal point
*   Detect obstacles in line of view
*   Convert obstacles and goal point into top down view
*   Create a path to goal point (A* Algorithm)
*   Create velocity and direction mavlink messages to send to rover










**Image processing for Static Image**

In [0]:
# image_processing.py
import numpy as np
import cv2
from matplotlib import pyplot as plt

# height and width of image
IMAGE_H = 750
IMAGE_W = 596

# read in image
image = cv2.imread('final2.jpg')

# create float for image and float for size of warped image
src = np.float32([[0, IMAGE_H], [IMAGE_W, IMAGE_H], [0, 0], [IMAGE_W, 0]])
dst = np.float32([[190, IMAGE_H], [410, IMAGE_H], [0, 0], [IMAGE_W, 0]])

# change perspective of image from src to dst
M = cv2.getPerspectiveTransform(src,dst)
Minv = cv2.getPerspectiveTransform(dst,src)

image = image[0:IMAGE_H, 0:IMAGE_W]
# filter out everything except obstacles via color thresholding. Obstacles were
# gray laptops, so filters everything out not in that range
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
lower_blue = np.array([50,10,160])
upper_blue = np.array([110,100,255])
mask = cv2.inRange(hsv, lower_blue, upper_blue)

# converts mask image with only the obstacles to top down view
warped_img = cv2.warpPerspective(mask, M, (IMAGE_W, IMAGE_H))

#res = cv2.bitwise_and(image, image, mask= mask)
#edged = cv2.Canny(warped_img, 75, 400)
